In [ ]:
import cv2

import treasureArea as ta
import webCamera as wc

class blockDetect:
    def __init__(self):
        self.treasureArea = ta.TreasureArea()
        self.webCamera = wc.WebCamera(0)
    
    def detectArea(self):
        pass

    def detectBlock(self):
        pass
    
    def __imageProcess(self, image):
        pass

    def __findPoints(self, image):
        pass

    def __findArea(self, point):
        pass

    def __findBlock(self, area):
        pass
